Import all required tools

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

%matplotlib inline

mp = ixmp.Platform()

<IPython.core.display.Javascript object>

This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [2]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "add_learning",
    "introducing the add_learning feature on MESSAGEix",
    keep_solution=False,
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

#scen.commit(comment="Introducing emissions and setting an upper bound")
#scen.set_as_default()
#scen.solve()
#scen.var("OBJ")["lvl"]

This Scenario has a solution, use `Scenario.remove_solution()` or `Scenario.clone(..., keep_solution=False)`


Call all data required by add_tech module

In [3]:
with open('learning_data.yaml','r') as stream:
    learning_data = yaml.safe_load(stream)

Define years vintage and active according to `first_active_year` and scenarios year

In [4]:
parameters = {}
for tech in list(learning_data.keys()):
    parameters.update({par: list(MESSAGE_ITEMS[par]['idx_sets']) for par in list(learning_data[tech])})
data = {par: [] for par in list(parameters.keys())}

Then create dataframe for each parameter

In [5]:
# Creating The Individual DataFrame
for tech, par_dict in learning_data.items():
    for par, par_data in par_dict.items():
        if not isinstance(par_data, Mapping):
            par_data = {'value': par_data, 'unit': '-'}
        if 'size' in parameters[par]:
            value = list(par_data['value'].values())
            kwargs = {'size': list(par_data['value'].keys())}
        else:
            value = par_data['value']
            kwargs = {}
        data[par].append(
                make_df(
                    par,
                    technology=tech,
                    value=value,
                    unit=par_data['unit'],
                    **kwargs
                ))

data = {k: pd.concat(v).reset_index(drop=True) for k, v in data.items()}

Creating basic dataframe to be filled in later

In [6]:
with pd.ExcelWriter('printed_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [15]:
list(data.keys())

['learning_par', 'eos_par', 'nbr_unit_ref', 'u_ref', 'u']

In [11]:
selected_data = data['u'][data['u']['technology'] == 'DACCS']

In [35]:
list(selected_data.columns)

['technology', 'size', 'value', 'unit']

In [29]:
list(set(selected_data['size']))

['large', 'small', 'medium']